In [1]:
from __future__ import print_function
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates as dates
from IPython.display import display
from pyspark.sql.functions import *

In [2]:
spark = SparkContext("local", "sqlContext")
sql = SQLContext(spark)

In [3]:
data = spark.textFile('D:/data/csv/energy.csv')
data = data.filter(lambda x: x.split(',')).map(lambda line: line.split(','))

In [4]:
df = sql.createDataFrame(data, ['datetime', 'kwh'])
df.show()

+-------------+-----------+
|     datetime|        kwh|
+-------------+-----------+
|1/1/2017 0:15|1659089.647|
|1/1/2017 0:30|1649854.573|
|1/1/2017 0:45|1677086.704|
|1/1/2017 1:00|1662763.999|
|1/1/2017 1:15| 1673310.19|
|1/1/2017 1:30|1654926.304|
|1/1/2017 1:45| 1645992.15|
|1/1/2017 2:00|1620745.714|
|1/1/2017 2:15|1638862.016|
|1/1/2017 2:30|1611652.086|
|1/1/2017 2:45|1602853.015|
|1/1/2017 3:00|1579083.006|
|1/1/2017 3:15|1588359.234|
|1/1/2017 3:30|1564959.423|
|1/1/2017 3:45|1538248.067|
|1/1/2017 4:00|1513071.791|
|1/1/2017 4:15|1514415.352|
|1/1/2017 4:30| 1493824.69|
|1/1/2017 4:45|1473551.875|
|1/1/2017 5:00| 1460212.15|
+-------------+-----------+
only showing top 20 rows



In [5]:
df = df.withColumn("date", to_timestamp("datetime", "M/d/yyyy H:mm"))
df.show()

+-------------+-----------+-------------------+
|     datetime|        kwh|               date|
+-------------+-----------+-------------------+
|1/1/2017 0:15|1659089.647|2017-01-01 00:15:00|
|1/1/2017 0:30|1649854.573|2017-01-01 00:30:00|
|1/1/2017 0:45|1677086.704|2017-01-01 00:45:00|
|1/1/2017 1:00|1662763.999|2017-01-01 01:00:00|
|1/1/2017 1:15| 1673310.19|2017-01-01 01:15:00|
|1/1/2017 1:30|1654926.304|2017-01-01 01:30:00|
|1/1/2017 1:45| 1645992.15|2017-01-01 01:45:00|
|1/1/2017 2:00|1620745.714|2017-01-01 02:00:00|
|1/1/2017 2:15|1638862.016|2017-01-01 02:15:00|
|1/1/2017 2:30|1611652.086|2017-01-01 02:30:00|
|1/1/2017 2:45|1602853.015|2017-01-01 02:45:00|
|1/1/2017 3:00|1579083.006|2017-01-01 03:00:00|
|1/1/2017 3:15|1588359.234|2017-01-01 03:15:00|
|1/1/2017 3:30|1564959.423|2017-01-01 03:30:00|
|1/1/2017 3:45|1538248.067|2017-01-01 03:45:00|
|1/1/2017 4:00|1513071.791|2017-01-01 04:00:00|
|1/1/2017 4:15|1514415.352|2017-01-01 04:15:00|
|1/1/2017 4:30| 1493824.69|2017-01-01 04

In [6]:
df1 = df.select(col('date'),col('kwh').cast("float"))
df1.show()

+-------------------+---------+
|               date|      kwh|
+-------------------+---------+
|2017-01-01 00:15:00|1659089.6|
|2017-01-01 00:30:00|1649854.6|
|2017-01-01 00:45:00|1677086.8|
|2017-01-01 01:00:00|1662764.0|
|2017-01-01 01:15:00|1673310.2|
|2017-01-01 01:30:00|1654926.2|
|2017-01-01 01:45:00|1645992.1|
|2017-01-01 02:00:00|1620745.8|
|2017-01-01 02:15:00|1638862.0|
|2017-01-01 02:30:00|1611652.1|
|2017-01-01 02:45:00|1602853.0|
|2017-01-01 03:00:00|1579083.0|
|2017-01-01 03:15:00|1588359.2|
|2017-01-01 03:30:00|1564959.4|
|2017-01-01 03:45:00|1538248.1|
|2017-01-01 04:00:00|1513071.8|
|2017-01-01 04:15:00|1514415.4|
|2017-01-01 04:30:00|1493824.8|
|2017-01-01 04:45:00|1473551.9|
|2017-01-01 05:00:00|1460212.1|
+-------------------+---------+
only showing top 20 rows



In [7]:
w = df1.groupBy(window(col('date'), "1 day")).agg(sum("kwh").alias("tot_kwh"))
res = w.select(w.window.start.cast("string").alias("start"), w.window.end.cast("string").alias("end"), "tot_kwh")
res.show()

+-------------------+-------------------+----------------+
|              start|                end|         tot_kwh|
+-------------------+-------------------+----------------+
|2017-07-02 01:00:00|2017-07-03 01:00:00|1.144756308125E8|
|2017-05-26 01:00:00|2017-05-27 01:00:00| 1.29195138375E8|
|2017-12-05 00:00:00|2017-12-06 00:00:00|   1.928584675E8|
|2017-07-27 01:00:00|2017-07-28 01:00:00|   1.339933815E8|
|2017-12-22 00:00:00|2017-12-23 00:00:00|    1.79060005E8|
|2017-02-26 00:00:00|2017-02-27 00:00:00|   1.462383915E8|
|2017-03-11 00:00:00|2017-03-12 00:00:00|    1.50081542E8|
|2017-03-24 00:00:00|2017-03-25 00:00:00|  1.6386290975E8|
|2017-04-24 01:00:00|2017-04-25 01:00:00|  1.5348801425E8|
|2017-05-17 01:00:00|2017-05-18 01:00:00| 1.48710276125E8|
|2017-08-25 01:00:00|2017-08-26 01:00:00|   1.483355985E8|
|2017-07-03 01:00:00|2017-07-04 01:00:00|  1.4192462875E8|
|2017-02-01 00:00:00|2017-02-02 00:00:00|  1.8600690625E8|
|2017-04-25 01:00:00|2017-04-26 01:00:00|    1.62076854E

In [8]:
df_rsmp = res.select('start', 'tot_kwh').orderBy('start', ascending=True)
df_rsmp.show()

+-------------------+---------------+
|              start|        tot_kwh|
+-------------------+---------------+
|2017-01-01 00:00:00|1.49023903375E8|
|2017-01-02 00:00:00| 1.6167699375E8|
|2017-01-03 00:00:00|1.82517014625E8|
|2017-01-04 00:00:00|1.88174253375E8|
|2017-01-05 00:00:00|1.89774616125E8|
|2017-01-06 00:00:00|1.95552006625E8|
|2017-01-07 00:00:00|1.83266428625E8|
|2017-01-08 00:00:00|   1.65937363E8|
|2017-01-09 00:00:00|1.96333224875E8|
|2017-01-10 00:00:00| 2.0010795475E8|
|2017-01-11 00:00:00|1.96311880875E8|
|2017-01-12 00:00:00|   1.94295819E8|
|2017-01-13 00:00:00|  1.963635015E8|
|2017-01-14 00:00:00|1.77155733375E8|
|2017-01-15 00:00:00| 1.6878190875E8|
|2017-01-16 00:00:00| 2.0378915475E8|
|2017-01-17 00:00:00| 2.1105399675E8|
|2017-01-18 00:00:00|    2.1266683E8|
|2017-01-19 00:00:00|2.11939849625E8|
|2017-01-20 00:00:00|2.09708213875E8|
+-------------------+---------------+
only showing top 20 rows



In [43]:
spark.stop()

## Credits & Links

https://medium.com/jbennetcodes/how-to-get-rid-of-loops-and-use-window-functions-in-pandas-or-spark-sql-907f274850e4